In [1]:
import os
import random
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from himalaya.backend import set_backend

from compare_variance_residual.residual import residual_method
from compare_variance_residual.simulation import generate_dataset
from compare_variance_residual.variance_partitioning import variance_partitioning

In [2]:
def get_path(alphas, cv, n_targets):
    path = os.path.join("results", f"targets={n_targets}", f"cv={cv}",
                        f"alphas={alphas.min()},{alphas.max()},{len(alphas)}", "varying training samples")
    os.makedirs(path, exist_ok=True)
    return path

In [3]:
def save_scores(samples_train, d_list, scalars, n_targets, n_samples, noise_target, cv, alphas):
    path = get_path(alphas, cv, n_targets)
    for n_samples_train in samples_train:
        print(n_samples_train)
        csv_path = os.path.join(path, f"scores_{n_samples_train}.csv")
        scores = pd.DataFrame()
        if os.path.exists(csv_path):
            print("skipping, already exists")
            continue
        Xs, Y = generate_dataset(d_list, scalars, n_targets, n_samples, noise_target)
        print("data generated")
        x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score = variance_partitioning(
            Xs, Y, n_samples_train, alphas, cv)
        print("variance partitioning done")

        scores["x1_score"] = x1_score
        scores["x2_score"] = x2_score
        scores["vp_joint_score"] = joint_score
        scores["vp_shared_score"] = x1_and_x2_score
        scores["vp_x1_unique_score"] = vp_x1_unique_score
        scores["vp_x2_unique_score"] = vp_x2_unique_score
        del x1_score, x2_score, joint_score, x1_and_x2_score, vp_x1_unique_score, vp_x2_unique_score

        _, _, x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score = residual_method(
            Xs, Y, n_samples_train, alphas, cv)
        print("residual method done")
        scores["rm_x2_to_x1_score"] = np.concatenate(
            [x2_to_x1_score, np.full(len(rm_x1_unique_score) - len(x2_to_x1_score), np.nan)])
        scores["rm_x1_to_x2_score"] = np.concatenate(
            [x1_to_x2_score, np.full(len(rm_x1_unique_score) - len(x1_to_x2_score), np.nan)])
        scores["rm_x1_unique_score"] = rm_x1_unique_score
        scores["rm_x2_unique_score"] = rm_x2_unique_score
        print(scores.head())
        del x2_to_x1_score, x1_to_x2_score, rm_x1_unique_score, rm_x2_unique_score
        del Xs, Y
        scores.to_csv(csv_path, index=False)

# Save scores for varying training samples

In [4]:
backend = set_backend("cupy", on_error="warn")
warnings.filterwarnings("ignore")
random.seed(42)

In [5]:
d_list = [100, 100, 100]
n_targets = 10000
n_samples_train = 10000
n_samples_test = 100
n_samples = n_samples_train + n_samples_test
scalars = [1 / 3, 1 / 3, 1 / 3]
noise_target = 0.1

cv = 10
alphas = np.logspace(-5, 5, 10)

In [6]:
samples_train = np.logspace(1, 6, 6).astype(int)

In [7]:
save_scores(samples_train, d_list, scalars, n_targets, n_samples, noise_target, cv, alphas)

10
data generated
variance partitioning done
   x1_score  x2_score  vp_joint_score  vp_shared_score  vp_x1_unique_score  \
0  0.000286  0.000390        0.012749        -0.012073            0.012359   
1  0.000221  0.000306        0.001964        -0.001437            0.001658   
2  0.000200  0.000244       -0.012319         0.012763           -0.012563   
3  0.000232  0.000153       -0.000929         0.001314           -0.001082   
4  0.000072  0.000163        0.001722        -0.001487            0.001558   

   vp_x2_unique_score  
0            0.012462  
1            0.001743  
2           -0.012519  
3           -0.001161  
4            0.001650  
residual method done
177
data generated
variance partitioning done
   x1_score  x2_score  vp_joint_score  vp_shared_score  vp_x1_unique_score  \
0  0.237087  0.226791        0.282811         0.181067            0.056020   
1  0.199447  0.200823        0.368740         0.031530            0.167917   
2  0.246920  0.231987        0.334350    

KeyboardInterrupt: 